In [1]:
from brian2 import *
import numpy as np
from itertools import groupby
from operator import itemgetter
import pickle

In [3]:
# network param.
N = 500;
N_E = N
N_readout = 400
N_extra_readout = 2

n_trials = 30

save_to = './data/Gaussian_population'
s = [0.25, 0.25 + 0.5] #stimulus conditions
s1 = s[0]
s2 = s[1]

# neuron param.
tau_mem = 20*ms
thresh = 20 *mV
V_reset = 10*mV
V_rest = 0*mV
tau_rp = 2*ms

eqs = '''
dv/dt = -(v-V_rest)/tau_mem : volt (unless refractory)
'''

D = 1.5 *ms # Synaptic delay


# synapse param.
J_E = 0.2*mV
J_EE = 10*J_E
J_I = J_EE

# preferred locations of 1st layer
theta_0 = np.linspace(0,1,N + 1)
theta_0 = theta_0[:-1] #remove last element (circular stimulus variable)

# preferred locations of readout layer neurons
readout_theta_0 = np.linspace(0,1,N_readout + 1)
readout_theta_0 = readout_theta_0[:-1]


max_input = 5000* Hz #Heuristically set
non_specific = 0.85
specific = 0.15

#controlls width of input modulation to input layer
w = 0.3 
# Spatial frequency of input layer neurons
f = 1 

# Poisson input rates to the input layer
# Baseline input is 0.85*5000 = 4250 Hz just enough to make neurons spike a few times, rest 0.15*5000 = 750 Hz is stimulus modulation
input_rates = lambda theta: (specific*exp(1/w*(cos(2*pi*f*(theta-theta_0)) - 1)) + non_specific)*max_input

In [4]:
#activity input to input layer

defaultclock.dt = 0.1 * ms
TC_vec = [list(input_rates(min(1,max(s[i],0)))) for i in range(len(s[:]))]
rate_vec = TimedArray(TC_vec, dt=500*ms)

In [5]:
for k in range(0,n_trials):
    # Get input to the first layer
    start_scope()
    P = PoissonGroup(N, rates='rate_vec(t,i)')
    MP_1 = SpikeMonitor(P)
    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_M_1 = MP_1.i
    spikes_t_M_1 = MP_1.t

    # Get baseline input activity to readout layer
    start_scope()
    defaultclock.dt = 0.1 * ms
    P = PoissonGroup(N_readout, rates=non_specific*max_input)
    MP = SpikeMonitor(P)
    run(1000*ms)

    # And keep a copy of those spikes
    spikes_i_readout_base = MP.i
    spikes_t_readout_base = MP.t
    
    
    ### Now we run the spiking network given the sampled inputs
    start_scope()
    defaultclock.dt = 0.1 * ms
    TC_neurons = NeuronGroup(N, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )


    readout_neurons = NeuronGroup(N_readout, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )
    
    TC_neurons.v = 10*(1 + rand(N_E))*mV # initial value


    readout_neurons.v = 10*(1 + rand(N_readout))*mV # initial value

    #Synapses input activity -> input layer
    SGG = SpikeGeneratorGroup(N, spikes_i_M_1, spikes_t_M_1)
    external_syn = Synapses(SGG, target=TC_neurons, on_pre='v += J_E')
    external_syn.connect(j='i') #connect 1-to-1

    #Synapses input layer -> readout layer (distance dependent EPSPs - in stimulus space)
    read_out_syn = Synapses(TC_neurons, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)

    read_out_syn.connect()
    FWHM = 1/N_readout;
    sigma = (FWHM/2) / sqrt(2*log(2));
    w_readout = (pi*FWHM)**2/(2*log(2))
    read_out_syn.epsp = 'exp(1/(w_readout)*(cos(2*pi*f*(1/N_E*i/f - 1/N_readout*j))-1))*J_EE'

    #Synapses baseline input to readout layer
    SGG_readout = SpikeGeneratorGroup(N_readout, spikes_i_readout_base, spikes_t_readout_base)
    external_syn_readout = Synapses(SGG_readout, target=readout_neurons, on_pre='v += J_E')
    external_syn_readout.connect(j='i') #connect 1-to-1

    #Winner take all style inhibitory synapses between readout layer (distance dependent, "nearer" -> less inhibition)
    read_out_syn_inhib = Synapses(readout_neurons, readout_neurons, 'ipsp : volt', on_pre='v += ipsp', delay = D)
    read_out_syn_inhib.connect()
    read_out_syn_inhib.ipsp = '-abs(sin(pi*(1/N_readout*i - 1/N_readout*j)))*J_I'
    
    
    spike_monitor_TC = SpikeMonitor(TC_neurons)

    spike_monitor_readout = SpikeMonitor(readout_neurons)
    


    run(1000*ms)
    # record spikes from 1st layer (the tuning curves)
    data_1 = spike_monitor_TC.get_states(['t','i','count'])
    # record spikes from the readout layer
    data_readout = spike_monitor_readout.get_states(['t','i','count'])
    

    with open(save_to + '/data_TC_layer_run_idx_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_1, save_file)

    with open(save_to + '/data_readout_layer_run_idx_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_readout, save_file)
        


WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]
